<h1 align="center">EECS E6893 Big Data Analytics - Homework Assignment 3</h1>
<h2 align="center">Name: Qi Wang</h2>
<h2 align="center">UNI: qw2261</h2>


This module is used to pull data from twitter API and send data to
Spark Streaming process using socket. It acts like a client of
twitter API and a server of spark streaming. It open a listening TCP
server socket, and listen to any connection from TCP client. After
a connection established, it send streaming data to it.


Usage:
  If used with dataproc:
    gcloud dataproc jobs submit pyspark --cluster <Cluster Name> twitterHTTPClient.py

  Make sure that you run this module before you run spark streaming process.
  Please remember stop the job on dataproc if you no longer want to stream data.

Todo:
  1. change the credentials to your own



In [2]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json

In [3]:
# credentials
# TODO: replace with your own credentials
ACCESS_TOKEN = '2852841985-5cFWzbWv4yWSi1BwSk6pwHU51t6vAEJLrZ3tdW1'     # your access token
ACCESS_SECRET = 'Ca2tIIWdkB4AdECHUFiaaY5LhVcYheciayQ6zsTbyvGld'    # your access token secret
CONSUMER_KEY = 'kuBnGga57JE6wn9HnfZSDu3tX'     # your API key
CONSUMER_SECRET = 'qQFTX5gGeY5ox3EgGxWhMCrqLVnZPnZ4JT6IHUjJElVDf0xfBD'  # your API secret key

# the tags to track
tags = ['#', 'bigdata', 'spark', 'ai', 'movie']

In [4]:
class TweetsListener(StreamListener):
    """
    tweets listener object
    """
    def __init__(self, csocket):
        self.client_socket = csocket
    def on_data(self, data):
        try:
            msg = json.loads( data )
            print('TEXT:{}\n'.format(msg['text']))
            self.client_socket.send( msg['text'].encode('utf-8') )
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
            return False
        # return True
    def on_error(self, status):
        print(status)
        return False

In [5]:
def sendData(c_socket, tags):
    """
    send data to socket
    """
    auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    twitter_stream.filter(track=tags,languages=['en'])

In [6]:
class twitter_client:
    def __init__(self, TCP_IP, TCP_PORT):
      self.s = s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      self.s.bind((TCP_IP, TCP_PORT))

    def run_client(self, tags):
      try:
        self.s.listen(1)
        while True:
          print("Waiting for TCP connection...")
          conn, addr = self.s.accept()
          print("Connected... Starting getting tweets.")
          sendData(conn,tags)
          conn.close()
      except KeyboardInterrupt:
        exit

In [ ]:
if __name__ == '__main__':
    client = twitter_client("localhost", 9001)
    client.run_client(tags)

Waiting for TCP connection...
Connected... Starting getting tweets.
TEXT:Good night for this movie. :3 https://t.co/WiasKr8Tss

TEXT:RT @Insideandria: nigga cooking a dinosaur https://t.co/ep3eufXENo

TEXT:RT @angies_code: We are going strong with 3 # trending WW! We will fight until you come back to us again!

2. #원호사랑해

5. #WONHOCOMEBACK

16…

TEXT:RT @Krazygioo: jus got my dick stuck in a pumpkin on this fine morning on halloween lol my life a movie

TEXT:Is Terminator a Halloween movie? 🤔

TEXT:RT @MarioAnthony__: There’s some people I just don’t believe would ever die. &amp; he was one of them 🙏🏾 https://t.co/Aixi7MzWBy

TEXT:RT @SamuelAAdams: why would Terrence Malick make a movie about resisting fascism in a language Americans can easily understand, I guess we'…

TEXT:RT @AIanHangover: Who else remembers this movie!? 🙈 https://t.co/GUDoFKrXes

TEXT:MY WORLDS ARE COLLIDING

TEXT:THE NITTY GRITTY EPICSODE 182! Newest Game Trailers + Movie Trailers &amp; REACTIONS! #Gaming #TalkSho

TEXT:The actually groundbreaking implications for AI technology is tilting Korean gamer nerds https://t.co/VnJVZTxmn5

TEXT:RT @Mhlengwe_E: @Olunloyo @Pfuxani @AinaTalks @ButiCollins @Mashele333 @HitekaniMag I totally agree with you. @AinaTalks  may have inadvert…

TEXT:Congratulations to Y9 leaders Kizzy &amp; Poppy. Amazing dance leadership, girls loved it! # Framearlpe https://t.co/6uGEYwHCAp

TEXT:wish i had someone w this mentality:/

TEXT:RT @morganherlocker: This is one of the most important pieces of software being developed in the open mapping world today. RapiD is cool en…

TEXT:RT @juicy_tayy: Tb to last Halloween when I took a selfie on this dudes phone cause he had a light up case n gave him my # in exchange for…

TEXT:RT @gayleallan7: @mom4boca @BlueVotr @kingmanmarie39 @IvankaTrump That is exactly the movie I thought of when I saw that pic!

TEXT:RT @maxvsyou: I wana go to a drive-in movie theater wtf

TEXT:Proud to be working on this project as well! 🥳🥳

TEXT:'What are 

TEXT:RT @RichardEudes: Sizing The Market Value Of Artificial Intelligence https://t.co/JtYYCCNLzI #bigdata, #businessanalytics #ba, #machinelear…

TEXT:My shirt is a nerdy reference to a movie that came out TWO decades ago. 😳 https://t.co/CaaxEmDMfZ

TEXT:RT @DineshDSouza: Hahaha! Totally made up story. No names! It’s all about “hushed conversations” as if the W. Post sits in on hushed conver…

TEXT:RT @JaylaG___: I be having intentions on watching movies but I always end up turning the movie on getting on my phone 😭

TEXT:RT @Rachelhodder: Want a scary movie to watch? 

Scariest movie ever 👻🎃🕷️🕸️
https://t.co/QhCrmtyM4u

#Halloween

TEXT:RT @dabi_hawks: ANYWAY if you type "Google drive" after a movie title in your search bar then you'll probably find pretty much any movie ou…

TEXT:RT @Roadmancode: Turkey is leading from the front in the war against the Vegans. https://t.co/aMhTconvpw

TEXT:RT @pornhubturks: Like - Retweet - Follow  
@pornhubturks

  🔻Link                             

TEXT:RT @winterpersists: Today was dress as an everyday superhero 🦸‍♀️ at school today. My kids know that I along with an army of badass moms ar…

TEXT:Its Halloween I guess so that means were gonna watch a Robert Undead movie

TEXT:A audio visual story
When The Wicked Gather: A Halloween Story..this a movie for yo ears..for my hip-hop heads that… https://t.co/Xgu15FOKt1

TEXT:It’s real, therefore that means it’s as bad as we can possibly conceive. 🙄

TEXT:What a beautiful movie ending! One of the best movie endings I have ever felt!
Loved this. If a driver has the cour… https://t.co/GZZzAc9Nft

TEXT:RT @MarioAnthony__: There’s some people I just don’t believe would ever die. &amp; he was one of them 🙏🏾 https://t.co/Aixi7MzWBy

TEXT:RT @AGWillliamBarr: "When Trump tweeted a photo of the Hero Dog receiving a Medal, it was very confusing for me. I didn't know if it was re…

TEXT:RT @juicy_tayy: Tb to last Halloween when I took a selfie on this dudes phone cause he had a light up case n g

TEXT:Father reveals that his autistic teenage daughter is locked in a 'cell' 24 hours a day with no physical contact and… https://t.co/FgcnIJXBpZ

TEXT:RT @WarpsiwaHUB: JUL-020
🔞 #วาร์ปสิวะ #JAV #WarpsiwaHUB
🎬 Full Movie https://t.co/cujjhaNE0x https://t.co/HyMTLWdc0T

TEXT:RT @juicy_tayy: Tb to last Halloween when I took a selfie on this dudes phone cause he had a light up case n gave him my # in exchange for…

TEXT:@boohoo Scary Movie 3

TEXT:RT @ronwangelin: Our 14th year of a family Halloween costume and our 3 boys wanted to go as their favorite new movie @SpiderVerse @pramsey3…

TEXT:When this gentile utilizes the spark to hinder you who is trying to pray, the kelipot are manifestly ruling over th… https://t.co/rjPkquZ5kC

TEXT:I'm gonna pop some more kettle corn and watch Shazam.

Doing these late ass reviews are fun!

My bedtime is usually… https://t.co/VWB5IngM8r

TEXT:RT @PowerAdos: @Chad_Boogie Kassi Lemmons, If you felt it was your duty to create confusing complexity in this

...

TEXT:RT @juicy_tayy: Tb to last Halloween when I took a selfie on this dudes phone cause he had a light up case n gave him my # in exchange for…

TEXT:@batrickpateman2 same. nobody prepared me, don't know the last time a movie fucked me up like that

TEXT:@SamHeughan Raised by a car dad. Two car brothers. Can’t wait to see this movie! I’ll be driving my classic yellow Ferrari...  😽😼

TEXT:I went to see the movie, The Lighthouse, today. I even dressed accordingly, but I left perplexed and confused as to… https://t.co/CLV3pTooMg

TEXT:📢 https://t.co/a5DoM5AOT6 The power and potential of AI in insurance claims

TEXT:RT @DashieXP: So I was watching child’s play 2 and never realized there was a fire ass beat potential in this movie! I had to add some drum…

TEXT:My favorite childhood Halloween movie was hocus-pocus https://t.co/OWcWfE8Ylu

TEXT:@krystacrez Perf halloween movie. Esp after an entire boottle of wine

TEXT:Every Girl Can Code and Girl Can Create: TSU-Apple coding initiative see

TEXT:RT @goodbeanalt: in all honesty I don’t care if your school is # 1 in engineering or STEM research. that stuff is for nerds. my school has…

TEXT:RT @Ronald_vanLoon: 10 Ways #AI Could Help Fight Climate Change 
 by @spelldotrun

 #BigData #MachineLearning #ArtificialIntelligence #ML #…

TEXT:Favorite part about halloween is that my favorite scary movie is on repeat 😍😍 Michael Myers !!

TEXT:@Ask_Spectrum Appreciate t/heads up however, as a 10 yr Spectrum customer am aware of t/above times..  However, due… https://t.co/VZFRk8xRWy

TEXT:@Millerita @RealJamesWoods 
What Halloween movie would be good tonight staring Mr.Woods?

TEXT:I’m ready for the corny Christmas movie marathons

TEXT:@DaddyCrowell1 Great movie!!!

TEXT:@Femtenks Cooler or that SSJ Goku from the cooler movie.

TEXT:RT @petridishes: horror movie where millennials manage to buy a house but it’s haunted by the ghosts of all the things millennials have kil…

TEXT:RT @etherealarthur: With all the hype surrounding the new

TEXT:@SpotifyCares Have an urgent issue with my @TuneCore release to @spotifyartists that needs immediate addressing for… https://t.co/hR2ldpFpIN

TEXT:Have you seen "The Greatest Showman"? This inspiring song became the anthem of the movie. I watched it tonight, mov… https://t.co/jHBJ76s1Dv

TEXT:RT @tariqnasheed: The fact that the producers of the Harriet movie crowbarred in a Black male villain character WHO DID NOT EXIST in any hi…

TEXT:RT @petridishes: horror movie where millennials manage to buy a house but it’s haunted by the ghosts of all the things millennials have kil…

TEXT:RT @lanyrila: for the past few years we're really focused about the gross  but can we set this film aside? let's talk about QUALITY, it may…

TEXT:RT @ajhinch: Thank you @astros fans for a memorable 2019 season. From standout player performances to a franchise record # of wins to clinc…

TEXT:RT @WarpsiwaHUB: DANDY-462 Natsume Eri
🔞 #วาร์ปสิวะ #JAV #WarpsiwaHUB
🎬 Full Movie https://t.co/TyjOw7lPqx https:

TEXT:RT @BessebBonWalker: #myart #atlasAU

- when you wanted to change the color of the your AI for fun, but as a result had fun only this AI

H…

TEXT:Now until midnight: Add some campy horror and sci-fi to your #Halloween with ETV's Creature Features movie marathon… https://t.co/F7zesEGfL3

TEXT:NSF Funds UCF Study to Explore AI Impact on Hospitality Industry https://t.co/FkfgAkcNUV

TEXT:Find out what makes a scary movie scary as @RealDeanCain counts down the scariest movies of all time TONIGHT at 8p… https://t.co/uiiZQoHYHz

TEXT:AI makes all, wiser.

*Short Lecture "You and I" on YouTube
From "7.Go Punk"
https://t.co/BGW6lgbavo… https://t.co/nXWf7fzr0p

TEXT:Brad, Grayson, and Jessie spend a quiet evening in with a classic horror movie marathon. Find out what happens when… https://t.co/fmWwIWYE5K

TEXT:Why Freddy Krueger is perhaps the most iconic horror movie villain
https://t.co/fanMD4fg1P https://t.co/MiirU5d615

TEXT:Our own Sameh Sabry takes Three Card Monte into the digital 

TEXT:RT @No_Safe_Spaces: SAN DIEGO: Screenings start tonight in El Cajon, CA. Get your tickets for No Safe Spaces this weekend --&gt;https://t.co/u…

TEXT:Join us for #AIExperience in Singapore on November 5th! Meet up with team DataRobot for live demos and insights. He… https://t.co/csIPgqHSvP

TEXT:RT @soshialism: [TUTORIAL] How to vote for Yoona &amp; Sooyoung @ Blue Dragon Film Awards

👉 https://t.co/PULQixlgwU

*Since you're required to…

TEXT:@vydija @IssabelCosplay I wish I could. More like I'm having a Snowy Halloween. It's like a reversed version of thi… https://t.co/DTHTeiJXK5

TEXT:BBS HAVING NO SPARK SYSTEM REALLY BITES RN TBH

TEXT:race # 9 - 10-3-4-2-  win - pays - $2.80 - $2.40 --

TEXT:Halloween movie and #bestofthe2010s nominee: “Black Swan.” https://t.co/tvtMRTMITN

TEXT:RT @ajhinch: Thank you @astros fans for a memorable 2019 season. From standout player performances to a franchise record # of wins to clinc…

TEXT:@alexandersallen The one year I made a jack-o-lantern